In [181]:
import spacy
import pandas as pd
from torchtext.data import Field, BucketIterator, TabularDataset
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import softmax

In [182]:
train = open("Info.txt", encoding = "utf8").read().lower().split('\n')

In [183]:
en = spacy.load('en_core_web_sm')

In [184]:
train

['your name is oliver',
 'you are 20 years old',
 'you have red hair',
 'you are male',
 'you live in bristol',
 'you make music',
 'you are a software engineer',
 'you are incredibly smart',
 'you are humble',
 'you are funny',
 "you don't change your bed sheets"]

In [185]:
words = pickle.load(open("6B.50_words.pkl", "rb"))

In [186]:
word2idx = pickle.load(open("6B.50_idx.pkl", "rb"))

In [187]:
def tokenize(text):
    return [word2idx[word] for word in text.split() if word in words]

In [188]:
def pad(tensor, length):
    t = np.zeros(length, dtype = np.int)
    tensor = tensor[:length]
    to_idx = tensor.shape[0]
    t[:to_idx] = tensor
    return t

In [189]:
data = np.array([pad(torch.tensor(tokenize(sent)).long(), 10) for sent in train])
data = torch.Tensor(data).long()

In [391]:
questions = open("Questions.txt", encoding = "utf8").read().lower().split('\n')

In [392]:
questions[0].split(',')

['is your name oliver-is your name danny-is your name jess', '1-0-0']

In [393]:
questions = [[segment.split('-') for segment in sample.split(',')] for sample in questions]

In [573]:
class MemoryBlock(nn.Module):
    def __init__(self, vocab_size, d_model, X):
        super().__init__()
        self.d_model = d_model
        
        self.embed = nn.Embedding(vocab_size, d_model)
        self.tokeys = nn.Linear(d_model, d_model)
        self.toqueries = nn.Linear(d_model, d_model)
        self.tovalues = nn.Linear(d_model, d_model)
        
        self.do = nn.Dropout(0.1)
        
        self.norm1 = nn.LayerNorm([d_model])
        
        self.encodedtomemory = nn.Linear(d_model, X)
        self.tomemory = nn.Linear(d_model, X)
        self.toresult = nn.Linear(X, 2)
        
        self.norm2 = nn.LayerNorm([X])
        
        self.memory = torch.rand(X, X)
        
    def forward(self, x, questions):
        #Keys Queries Values
        x = self.embed(x)
        x = self.do(x)
        keys = self.tokeys(x)
        queries = self.toqueries(x)
        values = self.tovalues(x)
        
        q = self.embed(questions)
        qkeys = self.tokeys(q)
        qqueries = self.toqueries(q)
        qvalues = self.tovalues(q)
        
        #Attention
        dot = torch.bmm(queries, keys.transpose(1, 2))/np.sqrt(self.d_model)
        attention = torch.bmm(softmax(dot, dim = 1), values)
        attention = self.norm1(attention + x)
        attention = self.encodedtomemory(torch.sum(torch.mean(attention, dim = 1), dim=0))
        attention = self.do(attention)
        attention = torch.unsqueeze(attention, 0)
        attention = torch.mm(attention.transpose(0,1), attention)
        
        #Update Memory
        tmp = torch.zeros(self.memory.shape).to(device)
        tmp[:,:] = self.memory.detach()
        self.memory = attention + tmp
        self.memory = self.norm2(self.memory)
        
        #Encode Question
        qdot = torch.mm(qqueries, qkeys.transpose(0, 1))/np.sqrt(self.d_model)
        qattention = torch.mm(softmax(qdot, dim = 1), qvalues)
        qattention = self.norm1(qattention + q)
        qattention = self.encodedtomemory(torch.mean(qattention, dim = 0))
        qattention = self.do(qattention)
        qattention = torch.unsqueeze(qattention, 0)
        result = self.toresult(torch.mm(qattention, self.memory))
        result = softmax(result, dim = 1)
        
        return result
        

In [574]:
device = "cuda" if torch.cuda.is_available() else "cpu"
mb = MemoryBlock(len(words), 10, 15).to(device)

In [575]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mb.parameters(), lr = 1e-4)

In [577]:
torch.autograd.set_detect_anomaly(True)
for e in range(100):
    for i,(q,v) in enumerate(questions):
        d = data[i]
        for j, statement in enumerate(q):
            s = np.array(pad(torch.tensor(tokenize(statement)).long(), 10))
            s = torch.Tensor(s).long().to(device)
            optimizer.zero_grad()

            label = torch.Tensor([int(v[j])]).long().to(device)
            out = mb(data.to(device), s).to(device)
            loss = criterion(out, label).to(device)
            loss.backward(retain_graph=True)
            optimizer.step()
            print(out)
            print(label)
            print(loss.item())

tensor([[0.1670, 0.8330]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.4145941734313965
tensor([[0.5342, 0.4658]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.659525454044342
tensor([[0.3433, 0.6567]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.8621164560317993
tensor([[0.4944, 0.5056]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.6875771880149841
tensor([[0.4984, 0.5016]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.6915927529335022
tensor([[0.7655, 0.2345]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.4624871015548706
tensor([[0.3674, 0.6326]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.8345324397087097
tensor([[0.4294, 0.5706]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.6250441074371338
tensor([[0.4573, 0.5427]], device='cuda:0', grad_

tensor([[0.0281, 0.9719]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
1.2724465131759644
tensor([[0.0317, 0.9683]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.3306848406791687
tensor([[0.2388, 0.7612]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.9880669713020325
tensor([[0.5172, 0.4828]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.676047682762146
tensor([[0.7832, 0.2168]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
1.0158617496490479
tensor([[0.8056, 0.1944]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
1.0447958707809448
tensor([[0.8116, 0.1884]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.4293433725833893
tensor([[0.4209, 0.5791]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.7753857374191284
tensor([[0.4724, 0.5276]], device='cuda:0', grad_

tensor([[0.4402, 0.5598]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.6350923180580139
tensor([[0.7528, 0.2472]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.47192901372909546
tensor([[0.4051, 0.5949]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.6027445793151855
tensor([[0.3537, 0.6463]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.8500728607177734
tensor([[0.5261, 0.4739]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.6673763990402222
tensor([[0.6738, 0.3262]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.8820270895957947
tensor([[0.8638, 0.1362]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
1.1217182874679565
tensor([[0.5948, 0.4052]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.6028153896331787
tensor([[0.4143, 0.5857]], device='cuda:0', gra

tensor([[0.6928, 0.3072]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.9043678641319275
tensor([[0.3505, 0.6495]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.5548158884048462
tensor([[0.6809, 0.3191]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.5285285711288452
tensor([[0.5308, 0.4692]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.7244412302970886
tensor([[0.4075, 0.5925]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.7899549007415771
tensor([[0.8857, 0.1143]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.380085825920105
tensor([[0.7053, 0.2947]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.9194124937057495
tensor([[0.0238, 0.9762]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.32627052068710327
tensor([[0.3801, 0.6199]], device='cuda:0', grad

tensor([[0.6491, 0.3509]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.5551278591156006
tensor([[0.3159, 0.6841]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.5258886814117432
tensor([[0.7057, 0.2943]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.9198307394981384
tensor([[0.8200, 0.1800]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.4234764873981476
tensor([[0.0399, 0.9601]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.33537667989730835
tensor([[0.0204, 0.9796]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
1.2836647033691406
tensor([[0.5910, 0.4090]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.6063172817230225
tensor([[0.1451, 0.8549]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.3999764621257782
tensor([[0.0895, 0.9105]], device='cuda:0', gra

...

tensor([[0.7195, 0.2805]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.9365468621253967
tensor([[0.2087, 0.7913]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.44370585680007935
tensor([[0.3169, 0.6831]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.8929375410079956
tensor([[0.0578, 0.9422]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
1.230125904083252
tensor([[0.3746, 0.6254]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.5756077170372009
tensor([[0.3101, 0.6899]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.9009901285171509
tensor([[0.4199, 0.5801]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.6162451505661011
tensor([[0.4512, 0.5488]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.7431292533874512
tensor([[0.3458, 0.6542]], device='cuda:0', grad

tensor([[0.4390, 0.5610]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.6339923143386841
tensor([[0.5255, 0.4745]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.7189568281173706
tensor([[0.4587, 0.5413]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.652745246887207
tensor([[0.6842, 0.3158]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.5258410573005676
tensor([[0.4926, 0.5074]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.7005773782730103
tensor([[0.8385, 0.1615]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
1.0878944396972656
tensor([[0.7901, 0.2099]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.44457924365997314
tensor([[0.5940, 0.4060]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.791511595249176
tensor([[0.2954, 0.7046]], device='cuda:0', grad_

tensor([[0.7457, 0.2543]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.4772982895374298
tensor([[0.5670, 0.4330]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.762410044670105
tensor([[0.8011, 0.1989]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
1.038935661315918
tensor([[0.6570, 0.3430]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.8623933792114258
tensor([[0.6118, 0.3882]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.5876031517982483
tensor([[0.4619, 0.5381]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.7319700717926025
tensor([[0.6221, 0.3779]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.822667121887207
tensor([[0.6128, 0.3872]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.5866767168045044
tensor([[0.3180, 0.6820]], device='cuda:0', grad_fn

tensor([[0.6004, 0.3996]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.5977908372879028
tensor([[0.2985, 0.7015]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.9148232936859131
tensor([[0.2779, 0.7221]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.4955275058746338
tensor([[0.4274, 0.5726]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.6231424808502197
tensor([[0.1968, 0.8032]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.4352128207683563
tensor([[0.2447, 0.7553]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.9807237982749939
tensor([[0.8248, 0.1752]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.42018866539001465
tensor([[0.6630, 0.3370]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.8693954944610596
tensor([[0.7078, 0.2922]], device='cuda:0', gra

tensor([[0.2565, 0.7435]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.47897306084632874
tensor([[0.2879, 0.7121]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.9275524616241455
tensor([[0.3758, 0.6242]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.8250769376754761
tensor([[0.6746, 0.3254]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.8829643726348877
tensor([[0.8367, 0.1633]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
1.0855120420455933
tensor([[0.6671, 0.3329]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([1], device='cuda:0')
0.8741554021835327
tensor([[0.5843, 0.4157]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.6123626828193665
tensor([[0.8456, 0.1544]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0], device='cuda:0')
0.4060959219932556
tensor([[0.7266, 0.2734]], device='cuda:0', gra